In [1]:
import scanpy as sc
import pandas as pd 
import numpy as np
import sys
import matplotlib.pyplot as plt
import gc
import anndata
import glob
from multiprocessing import Pool
import os
from sklearn import metrics
import matplotlib.pyplot as plt
from adjustText import adjust_text

/home/workspace/Aux_Envir/Python_BRI_Figures/lib/python3.11/site-packages/anndata/utils.py:429: FutureWarning: Importing read_csv from `anndata` is deprecated. Import anndata.io.read_csv instead.
  warnings.warn(msg, FutureWarning)
/home/workspace/Aux_Envir/Python_BRI_Figures/lib/python3.11/site-packages/anndata/utils.py:429: FutureWarning: Importing read_excel from `anndata` is deprecated. Import anndata.io.read_excel instead.
  warnings.warn(msg, FutureWarning)
/home/workspace/Aux_Envir/Python_BRI_Figures/lib/python3.11/site-packages/anndata/utils.py:429: FutureWarning: Importing read_hdf from `anndata` is deprecated. Import anndata.io.read_hdf instead.
  warnings.warn(msg, FutureWarning)
/home/workspace/Aux_Envir/Python_BRI_Figures/lib/python3.11/site-packages/anndata/utils.py:429: FutureWarning: Importing read_loom from `anndata` is deprecated. Import anndata.io.read_loom instead.
  warnings.warn(msg, FutureWarning)
/home/workspace/Aux_Envir/Python_BRI_Figures/lib/python3.11/site-p

In [2]:
def gen_mpl_labels(adata, groupby, exclude=(), ax=None, adjust_kwargs=None, text_kwargs=None, offset=(0.05, 0.05)):
    if adjust_kwargs is None:
        adjust_kwargs = {
            "arrowprops": {"arrowstyle": "--->", "color": 'r'}, 
            "expand_points": (1.2, 1.2), 
            "force_points": 0.5
        }
    if text_kwargs is None:
        text_kwargs = {}

    highest_density_points = {}

    for g, g_idx in adata.obs.groupby(groupby).groups.items():
        if g in exclude:
            continue
        points = adata[g_idx].obsm["X_umap"]
        hist, xedges, yedges = np.histogram2d(points[:, 0], points[:, 1], bins=[50, 50])
        max_bin = np.unravel_index(np.argmax(hist), hist.shape)
        highest_density_point = [
            (xedges[max_bin[0]] + xedges[max_bin[0]+1])/2 + offset[0], 
            (yedges[max_bin[1]] + yedges[max_bin[1]+1])/2 + offset[1]
        ]
        highest_density_points[g] = highest_density_point

    if ax is None:
        ax = plt.gca()

    texts = [
        ax.text(x=x, y=y, s=k, **text_kwargs) for k, (x, y) in highest_density_points.items()
    ]

    adjust_text(texts, **adjust_kwargs)

In [3]:
adata=sc.read_h5ad("adata.h5ad")
labels=pd.read_csv("labels.csv",index_col=0)

df=adata.obs.merge(labels, how='inner',left_index=True, right_index=True)

adata=adata[df.index]

adata.obs=df

FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = 'adata.h5ad', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [ ]:
adata.X[1:20,1:20].todense()

# CD4 T

In [ ]:
UMAP_Cord=pd.read_csv("/home/jupyter/Maxim_dataset_followup/GEX_HTO_processed/cd4_t_cells/cd4_umap.csv",index_col=0)

In [ ]:
adata_subset=adata[UMAP_Cord.index]

In [ ]:
adata_subset.obsm['X_umap'] =np.array(UMAP_Cord)

In [ ]:
adata_subset=adata_subset[adata_subset.obs[adata_subset.obs['AIFI_L3'].str.contains("CD4", case=False, na=False)].index]

In [ ]:
with plt.rc_context({"figure.figsize": (5, 5), "figure.dpi": 500, "figure.frameon": False}):
    ax=sc.pl.umap(adata_subset, color=['AIFI_L3'], size=0.5,show=False,legend_loc=None,ncols=1,frameon=False)
    gen_mpl_labels(
        adata_subset,
        "AIFI_L3",
        exclude=(""),  
        ax=ax,
        adjust_kwargs=dict(arrowprops=dict(arrowstyle='-', color='black')),
        text_kwargs=dict(fontsize=5),)
    fig = ax.get_figure()
    fig.tight_layout()

In [ ]:
with plt.rc_context({"figure.figsize": (5, 5), "figure.dpi": 500, "figure.frameon": False}):
    ax=sc.pl.umap(adata_subset, color=['Origional_Labels_L2'], size=0.5,show=False,legend_loc=None,ncols=1,frameon=False)
    gen_mpl_labels(
        adata_subset,
        "Origional_Labels_L2",
        exclude=(""),  
        ax=ax,
        adjust_kwargs=dict(arrowprops=dict(arrowstyle='-', color='black')),
        text_kwargs=dict(fontsize=5),)
    fig = ax.get_figure()
    fig.tight_layout()

# Conventional CD8 T

In [ ]:
UMAP_Cord=pd.read_csv("/home/jupyter/Maxim_dataset_followup/GEX_HTO_processed/conventional_cd8_t_cells/conventional_cd8_umap.csv",index_col=0)

In [ ]:
adata_subset=adata[UMAP_Cord.index]

In [ ]:
adata_subset.obsm['X_umap'] =np.array(UMAP_Cord)

In [ ]:
adata_subset=adata_subset[adata_subset.obs[adata_subset.obs['AIFI_L3'].str.contains("CD8|NK", case=False, na=False)].index]

In [ ]:
adata_subset.obs['AIFI_L3'].value_counts()

In [ ]:
with plt.rc_context({"figure.figsize": (5, 5), "figure.dpi": 500, "figure.frameon": False}):
    ax=sc.pl.umap(adata_subset, color=['AIFI_L3'], size=0.5,show=False,legend_loc=None,ncols=1,frameon=False)
    gen_mpl_labels(
        adata_subset,
        "AIFI_L3",
        exclude=(""),  
        ax=ax,
        adjust_kwargs=dict(arrowprops=dict(arrowstyle='-', color='black')),
        text_kwargs=dict(fontsize=5),)
    fig = ax.get_figure()
    fig.tight_layout()

In [ ]:
with plt.rc_context({"figure.figsize": (5, 5), "figure.dpi": 500, "figure.frameon": False}):
    ax=sc.pl.umap(adata_subset, color=['Origional_Labels_L2'], size=0.5,show=False,legend_loc=None,ncols=1,frameon=False)
    gen_mpl_labels(
        adata_subset,
        "Origional_Labels_L2",
        exclude=(""),  
        ax=ax,
        adjust_kwargs=dict(arrowprops=dict(arrowstyle='-', color='black')),
        text_kwargs=dict(fontsize=5),)
    fig = ax.get_figure()
    fig.tight_layout()

# CD4 T helper

In [ ]:
UMAP_Cord=pd.read_csv("/home/jupyter/Maxim_dataset_followup/GEX_HTO_processed/cd4_t_helper_memory_cells/cd4_helper_memory_umap.csv",index_col=0)

In [ ]:
adata_subset=adata[UMAP_Cord.index]

In [ ]:
adata_subset.obsm['X_umap'] =np.array(UMAP_Cord)

In [ ]:
adata_subset=adata_subset[adata_subset.obs[adata_subset.obs['AIFI_L3'].str.contains("CD4", case=False, na=False)].index]

In [ ]:
adata_subset.obs['AIFI_L3'].value_counts()

In [ ]:
with plt.rc_context({"figure.figsize": (5, 5), "figure.dpi": 500, "figure.frameon": False}):
    ax=sc.pl.umap(adata_subset, color=['AIFI_L3'], size=0.5,show=False,legend_loc=None,ncols=1,frameon=False)
    gen_mpl_labels(
        adata_subset,
        "AIFI_L3",
        exclude=(""),  
        ax=ax,
        adjust_kwargs=dict(arrowprops=dict(arrowstyle='-', color='black')),
        text_kwargs=dict(fontsize=5),)
    fig = ax.get_figure()
    fig.tight_layout()

In [ ]:
with plt.rc_context({"figure.figsize": (5, 5), "figure.dpi": 500, "figure.frameon": False}):
    ax=sc.pl.umap(adata_subset, color=['Origional_Labels_L2'], size=0.5,show=False,legend_loc=None,ncols=1,frameon=False)
    gen_mpl_labels(
        adata_subset,
        "Origional_Labels_L2",
        exclude=(""),  
        ax=ax,
        adjust_kwargs=dict(arrowprops=dict(arrowstyle='-', color='black')),
        text_kwargs=dict(fontsize=5),)
    fig = ax.get_figure()
    fig.tight_layout()

# gd T

In [ ]:
UMAP_Cord=pd.read_csv("/home/jupyter/Maxim_dataset_followup/GEX_HTO_processed/gd_t_cells/gd_t_cells_umap.csv",index_col=0)

In [ ]:
adata_subset=adata[UMAP_Cord.index]

In [ ]:
adata_subset.obsm['X_umap'] =np.array(UMAP_Cord)

In [ ]:
adata_subset=adata_subset[adata_subset.obs[adata_subset.obs['AIFI_L3'].str.contains("gd", case=False, na=False)].index]

In [ ]:
adata_subset.obs['AIFI_L3'].value_counts()

In [ ]:
with plt.rc_context({"figure.figsize": (5, 5), "figure.dpi": 500, "figure.frameon": False}):
    ax=sc.pl.umap(adata_subset, color=['AIFI_L3'], size=0.5,show=False,legend_loc=None,ncols=1,frameon=False)
    gen_mpl_labels(
        adata_subset,
        "AIFI_L3",
        exclude=(""),  
        ax=ax,
        adjust_kwargs=dict(arrowprops=dict(arrowstyle='-', color='black')),
        text_kwargs=dict(fontsize=5),)
    fig = ax.get_figure()
    fig.tight_layout()

In [ ]:
with plt.rc_context({"figure.figsize": (5, 5), "figure.dpi": 500, "figure.frameon": False}):
    ax=sc.pl.umap(adata_subset, color=['Origional_Labels_L2'], size=0.5,show=False,legend_loc=None,ncols=1,frameon=False)
    gen_mpl_labels(
        adata_subset,
        "Origional_Labels_L2",
        exclude=(""),  
        ax=ax,
        adjust_kwargs=dict(arrowprops=dict(arrowstyle='-', color='black')),
        text_kwargs=dict(fontsize=5),)
    fig = ax.get_figure()
    fig.tight_layout()

# MAIT

In [ ]:
UMAP_Cord=pd.read_csv("/home/jupyter/Maxim_dataset_followup/GEX_HTO_processed/mait_cells/mait_cells_umap.csv",index_col=0)

In [ ]:
adata_subset=adata[UMAP_Cord.index]

In [ ]:
adata_subset.obsm['X_umap'] =np.array(UMAP_Cord)

In [ ]:
adata_subset=adata_subset[adata_subset.obs[adata_subset.obs['AIFI_L3'].str.contains("MAIT", case=False, na=False)].index]

In [ ]:
with plt.rc_context({"figure.figsize": (5, 5), "figure.dpi": 500, "figure.frameon": False}):
    ax=sc.pl.umap(adata_subset, color=['AIFI_L3'], size=0.5,show=False,legend_loc=None,ncols=1,frameon=False)
    gen_mpl_labels(
        adata_subset,
        "AIFI_L3",
        exclude=(""),  
        ax=ax,
        adjust_kwargs=dict(arrowprops=dict(arrowstyle='-', color='black')),
        text_kwargs=dict(fontsize=5),)
    fig = ax.get_figure()
    fig.tight_layout()

In [ ]:
with plt.rc_context({"figure.figsize": (5, 5), "figure.dpi": 500, "figure.frameon": False}):
    ax=sc.pl.umap(adata_subset, color=['Origional_Labels_L2'], size=0.5,show=False,legend_loc=None,ncols=1,frameon=False)
    gen_mpl_labels(
        adata_subset,
        "Origional_Labels_L2",
        exclude=(""),  
        ax=ax,
        adjust_kwargs=dict(arrowprops=dict(arrowstyle='-', color='black')),
        text_kwargs=dict(fontsize=5),)
    fig = ax.get_figure()
    fig.tight_layout()

# B T

In [ ]:
UMAP_Cord=pd.read_csv("/home/jupyter/Maxim_dataset_followup/GEX_HTO_processed/b_cells/b_cells_umap.csv",index_col=0)

In [ ]:
adata_subset=adata[UMAP_Cord.index]

In [ ]:
adata_subset.obsm['X_umap'] =np.array(UMAP_Cord)

In [ ]:
adata_subset=adata_subset[adata_subset.obs[adata_subset.obs['AIFI_L3'].str.contains("B cell|Plasma", case=False, na=False)].index]

In [ ]:
adata_subset.obs['AIFI_L3'].value_counts()

In [ ]:
with plt.rc_context({"figure.figsize": (5, 5), "figure.dpi": 500, "figure.frameon": False}):
    ax=sc.pl.umap(adata_subset, color=['AIFI_L3'], size=0.5,show=False,legend_loc=None,ncols=1,frameon=False)
    gen_mpl_labels(
        adata_subset,
        "AIFI_L3",
        exclude=(""),  
        ax=ax,
        adjust_kwargs=dict(arrowprops=dict(arrowstyle='-', color='black')),
        text_kwargs=dict(fontsize=5),)
    fig = ax.get_figure()
    fig.tight_layout()

In [ ]:
with plt.rc_context({"figure.figsize": (5, 5), "figure.dpi": 500, "figure.frameon": False}):
    ax=sc.pl.umap(adata_subset, color=['Origional_Labels_L2'], size=0.5,show=False,legend_loc=None,ncols=1,frameon=False)
    gen_mpl_labels(
        adata_subset,
        "Origional_Labels_L2",
        exclude=(""),  
        ax=ax,
        adjust_kwargs=dict(arrowprops=dict(arrowstyle='-', color='black')),
        text_kwargs=dict(fontsize=5),)
    fig = ax.get_figure()
    fig.tight_layout()

# NK

In [ ]:
UMAP_Cord=pd.read_csv("/home/jupyter/Maxim_dataset_followup/GEX_HTO_processed/nk_cells/nk_cells_umap.csv",index_col=0)

In [ ]:
adata_subset=adata[UMAP_Cord.index]

In [ ]:
adata_subset.obsm['X_umap'] =np.array(UMAP_Cord)

In [ ]:
adata_subset=adata_subset[adata_subset.obs[adata_subset.obs['AIFI_L3'].str.contains("NK|ILC", case=False, na=False)].index]

In [ ]:
adata_subset.obs['AIFI_L3'].value_counts()

In [ ]:
with plt.rc_context({"figure.figsize": (5, 5), "figure.dpi": 500, "figure.frameon": False}):
    ax=sc.pl.umap(adata_subset, color=['AIFI_L3'], size=0.5,show=False,legend_loc=None,ncols=1,frameon=False)
    gen_mpl_labels(
        adata_subset,
        "AIFI_L3",
        exclude=(""),  
        ax=ax,
        adjust_kwargs=dict(arrowprops=dict(arrowstyle='-', color='black')),
        text_kwargs=dict(fontsize=5),)
    fig = ax.get_figure()
    fig.tight_layout()

In [ ]:
with plt.rc_context({"figure.figsize": (5, 5), "figure.dpi": 500, "figure.frameon": False}):
    ax=sc.pl.umap(adata_subset, color=['Origional_Labels_L2'], size=0.5,show=False,legend_loc=None,ncols=1,frameon=False)
    gen_mpl_labels(
        adata_subset,
        "Origional_Labels_L2",
        exclude=(""),  
        ax=ax,
        adjust_kwargs=dict(arrowprops=dict(arrowstyle='-', color='black')),
        text_kwargs=dict(fontsize=5),)
    fig = ax.get_figure()
    fig.tight_layout()

# Myeloid

In [ ]:
UMAP_Cord=pd.read_csv("/home/jupyter/Maxim_dataset_followup/GEX_HTO_processed/myeloid_cells/myeloid_cells_umap.csv",index_col=0)

In [ ]:
adata_subset=adata[UMAP_Cord.index]

In [ ]:
adata_subset.obsm['X_umap'] =np.array(UMAP_Cord)

In [ ]:
adata_subset=adata_subset[adata_subset.obs[adata_subset.obs['AIFI_L3'].str.contains("mono|DC", case=False, na=False)].index]

In [ ]:
with plt.rc_context({"figure.figsize": (5, 5), "figure.dpi": 500, "figure.frameon": False}):
    ax=sc.pl.umap(adata_subset, 
                  color=['AIFI_L3'], 
                  size=0.5,
                  show=False,
                  legend_loc=None,
                  ncols=1,
                  frameon=False)
    gen_mpl_labels(
        adata_subset,
        "AIFI_L3",
        exclude=(""),  
        ax=ax,
        adjust_kwargs=dict(arrowprops=dict(arrowstyle='-', color='black')),
        text_kwargs=dict(fontsize=5),)
    fig = ax.get_figure()
    fig.tight_layout()

In [ ]:
with plt.rc_context({"figure.figsize": (5, 5), "figure.dpi": 500, "figure.frameon": False}):
    ax=sc.pl.umap(adata_subset, color=['Origional_Labels_L2'], size=0.5,show=False,legend_loc=None,ncols=1,frameon=False)
    gen_mpl_labels(
        adata_subset,
        "Origional_Labels_L2",
        exclude=(""),  
        ax=ax,
        adjust_kwargs=dict(arrowprops=dict(arrowstyle='-', color='black')),
        text_kwargs=dict(fontsize=5),)
    fig = ax.get_figure()
    fig.tight_layout()